In [1]:
import sys
sys.path.append("/kaggle/input/torch-vision-references/references")

In [2]:
!pip install pycocotools
!pip install GPUtil

In [3]:
import os
from collections import namedtuple
import time
from torch.nn import functional as F
from tqdm import tqdm

from torch import nn
import torch as t
import torchvision

In [4]:
from torch.utils.data import DataLoader
from skimage import transform as sktsf
from torchvision import transforms as tvtsf

In [5]:
from torchvision.datasets import VOCDetection
import numpy as np
from numpy import random

import pandas as pd
import cv2
import os
import os.path as osp
import time
import random
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.transforms.functional as FT

from math import sqrt
from sklearn import preprocessing 
from torch.utils.data import DataLoader, Dataset

import torch
from torchvision import transforms
import types
import torchvision
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from PIL import Image, ImageDraw

In [6]:
dataset = VOCDetection(root=r'/kaggle/input/d/tungbinhthuong/pascal-voc-2012/',year ='2012', image_set='train', download=False)

test_dataset = VOCDetection(root=r'/kaggle/input/d/tungbinhthuong/pascal-voc-2012/', year = '2012',image_set='val', download=False)

In [7]:
def image_dict(split):
    image_dict = {}
    if split == "train":
        for image_info in dataset:
            image = image_info[0]
            image_id = image_info[1]['annotation']['filename'].split(".")[0]
            image_dict[image_id] = image
    if split == "test":
        for image_info in test_dataset:
            image = image_info[0]
            image_id = image_info[1]['annotation']['filename'].split(".")[0]
            image_dict[image_id] = image
    return image_dict

In [8]:
train_image_dict = image_dict("train")
test_image_dict = image_dict("test")

In [9]:
class Parser(object):

    def __init__(self,image):

        self.image = image
        self.annotations = self.image[1]['annotation']
        self.objects = self.annotations["object"]
       
        # image id 
        self.image_id = self.annotations['filename']
        # names of the classes contained in the xml file
        self.names = self._get_names()
        # coordinates of the bounding boxes
        self.boxes = self._get_bndbox()
        # difficulties
        self.difficulties = self._get_difficulties()

    def _get_names(self):

        names = []
        for obj in self.objects:
            name = obj['name']
            names.append(name)
        return np.array(names)

    def _get_bndbox(self):

        boxes = []
        for obj in self.objects:
            coordinates = []
            bndbox = obj["bndbox"]
            coordinates.append(np.int32(bndbox["xmin"]))
            coordinates.append(np.int32(bndbox["ymin"]))
            coordinates.append(np.int32(bndbox["xmax"]))
            coordinates.append(np.int32(bndbox["ymax"]))
            boxes.append(coordinates)

        return np.array(boxes)
    
    def _get_difficulties(self):
        difficulties = []
        for obj in self.objects:
            difficult = obj['difficult']
            difficulties.append(difficult)
        return np.array(difficult)

In [10]:
def inverse_normalize(img):
    if opt.caffe_pretrain:
        img = img + (np.array([122.7717, 115.9465, 102.9801]).reshape(3, 1, 1))
        return img[::-1, :, :]
    # approximate un-normalize for visualize
    return (img * 0.225 + 0.45).clip(min=0, max=1) * 255


def pytorch_normalze(img):
    """
    https://github.com/pytorch/vision/issues/223
    return appr -1~1 RGB
    """
    normalize = tvtsf.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
    img = normalize(t.from_numpy(img))
    return img.numpy()


def caffe_normalize(img):
    """
    return appr -125-125 BGR
    """
    img = img[[2, 1, 0], :, :]  # RGB-BGR
    img = img * 255
    mean = np.array([122.7717, 115.9465, 102.9801]).reshape(3, 1, 1)
    img = (img - mean).astype(np.float32, copy=True)
    return img


def preprocess(img, min_size=600, max_size=1000):
    """Preprocess an image for feature extraction.

    The length of the shorter edge is scaled to :obj:`self.min_size`.
    After the scaling, if the length of the longer edge is longer than
    :param min_size:
    :obj:`self.max_size`, the image is scaled to fit the longer edge
    to :obj:`self.max_size`.

    After resizing the image, the image is subtracted by a mean image value
    :obj:`self.mean`.

    Args:
        img (~numpy.ndarray): An image. This is in CHW and RGB format.
            The range of its value is :math:`[0, 255]`.

    Returns:
        ~numpy.ndarray: A preprocessed image.

    """
    C, H, W = img.shape
    scale1 = min_size / min(H, W)
    scale2 = max_size / max(H, W)
    scale = min(scale1, scale2)
    img = img / 255.
    img = sktsf.resize(img, (C, H * scale, W * scale), mode='reflect',anti_aliasing=False)
    # both the longer and shorter should be less than
    # max_size and min_size
    if opt['caffe_pretrain']:
        normalize = caffe_normalize
    else:
        normalize = pytorch_normalze
    return normalize(img)


class Transform(object):

    def __init__(self, min_size=600, max_size=1000):
        self.min_size = min_size
        self.max_size = max_size

    def __call__(self, in_data):
        img, bbox, label = in_data
        _, H, W = img.shape
        img = preprocess(img, self.min_size, self.max_size)
        _, o_H, o_W = img.shape
        scale = o_H / H
        bbox = util.resize_bbox(bbox, (H, W), (o_H, o_W))

        # horizontally flip
        img, params = util.random_flip(
            img, x_random=True, return_param=True)
        bbox = util.flip_bbox(
            bbox, (o_H, o_W), x_flip=params['x_flip'])

        return img, bbox, label, scale

In [11]:
def files_to_df(dataset):
    
    """"Return pandas dataframe from list of XML files."""
    
    names = []
    boxes = []
    image_id = []
    difficulties = []
    for image in dataset:
        parser = Parser(image)
        names.extend(parser.names)
        boxes.extend(parser.boxes)
        image_id.extend([parser.image_id] * len(parser.names))
        difficulties.extend([parser.difficulties])

        # xml_path.extend([xml.xml_file] * len(xml.names))
        # img_path.extend([xml.img_path] * len(xml.names))
    a = {"image_id": image_id,
         "names": names,
         "boxes": boxes,
         "difficulties": difficulties}
    
    df = pd.DataFrame.from_dict(a, orient='index')
    df = df.transpose()
    
    return df

df_train = files_to_df(dataset)
df_test = files_to_df(test_dataset)

In [12]:
def eval(dataloader, faster_rcnn, test_num=10000):
    pred_bboxes, pred_labels, pred_scores = list(), list(), list()
    gt_bboxes, gt_labels, gt_difficults = list(), list(), list()
    for ii, (imgs, sizes, gt_bboxes_, gt_labels_, gt_difficults_) in tqdm(enumerate(dataloader)):
        sizes = [sizes[0][0].item(), sizes[1][0].item()]
        pred_bboxes_, pred_labels_, pred_scores_ = faster_rcnn.predict(imgs, [sizes])
        gt_bboxes += list(gt_bboxes_.numpy())
        gt_labels += list(gt_labels_.numpy())
        gt_difficults += list(gt_difficults_.numpy())
        pred_bboxes += pred_bboxes_
        pred_labels += pred_labels_
        pred_scores += pred_scores_
        if ii == test_num: break

    result = eval_detection_voc(
        pred_bboxes, pred_labels, pred_scores,
        gt_bboxes, gt_labels, gt_difficults,
        use_07_metric=True)
    return result

In [13]:
# remove .jpg extension from image_id 
df_train['img_id'] = df_train['image_id'].apply(lambda x:x.split('.')).map(lambda x:x[0])
df_train.drop(columns=['image_id'], inplace=True)

df_test['img_id'] = df_test['image_id'].apply(lambda x:x.split('.')).map(lambda x:x[0])
df_test.drop(columns=['image_id'], inplace=True)

In [14]:
# classes need to be in int form so we use LabelEncoder for this task
enc = preprocessing.LabelEncoder()
df_train['labels'] = enc.fit_transform(df_train['names'])
df_train['labels'] = np.stack(df_train['labels'][i]+1 for i in range(len(df_train['labels'])))

df_test['labels'] = enc.fit_transform(df_test['names'])
df_test['labels'] = np.stack(df_test['labels'][i]+1 for i in range(len(df_test['labels'])))

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3472: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [15]:
# make dictionary for class objects so we can call objects by their keys.
rev_label_map= {0: 'background',
                1:'aeroplane',
              2:'bicycle',
              3:'bird',
              4:'boat',
              5:'bottle',
              6:'bus',
              7:'car',
              8:'cat',
              9:'chair',
              10:'cow',
              11:'diningtable',
              12:'dog',
              13:'horse',
              14:'motorbike',
              15:'person',
              16:'pottedplant',
              17:'sheep',
              18:'sofa',
              19:'train',
              20:'tvmonitor'}

label_map = {v: k for k, v in rev_label_map.items()}
label_map

{'background': 0,
 'aeroplane': 1,
 'bicycle': 2,
 'bird': 3,
 'boat': 4,
 'bottle': 5,
 'bus': 6,
 'car': 7,
 'cat': 8,
 'chair': 9,
 'cow': 10,
 'diningtable': 11,
 'dog': 12,
 'horse': 13,
 'motorbike': 14,
 'person': 15,
 'pottedplant': 16,
 'sheep': 17,
 'sofa': 18,
 'train': 19,
 'tvmonitor': 20}

In [16]:
# bounding box coordinates point need to be in separate columns

df_train['xmin'] = -1
df_train['ymin'] = -1
df_train['xmax'] = -1
df_train['ymax'] = -1

df_train[['xmin','ymin','xmax','ymax']]=np.stack(df_train['boxes'][i] for i in range(len(df_train['boxes'])))

df_train.drop(columns=['boxes'], inplace=True)
df_train['xmin'] = df_train['xmin'].astype(np.float32)
df_train['ymin'] = df_train['ymin'].astype(np.float32)
df_train['xmax'] = df_train['xmax'].astype(np.float32)
df_train['ymax'] = df_train['ymax'].astype(np.float32)
df_train['difficulties'] = df_train['difficulties'].astype(np.float32)

df_test['xmin'] = -1
df_test['ymin'] = -1
df_test['xmax'] = -1
df_test['ymax'] = -1

df_test[['xmin','ymin','xmax','ymax']]=np.stack(df_test['boxes'][i] for i in range(len(df_test['boxes'])))

df_test.drop(columns=['boxes'], inplace=True)
df_test['xmin'] = df_test['xmin'].astype(np.float32)
df_test['ymin'] = df_test['ymin'].astype(np.float32)
df_test['xmax'] = df_test['xmax'].astype(np.float32)
df_test['ymax'] = df_test['ymax'].astype(np.float32)
df_test['difficulties'] = df_test['difficulties'].astype(np.float32)

In [17]:
df_train.drop(columns=['names'], inplace=True)
df_test.drop(columns=['names'], inplace=True)

df_test.head()

,difficulties,img_id,labels,xmin,ymin,xmax,ymax
0,0.0,2008_000002,20,34.0,11.0,448.0,293.0
1,0.0,2008_000003,19,46.0,11.0,500.0,333.0
2,0.0,2008_000003,15,62.0,190.0,83.0,243.0
3,0.0,2008_000007,4,1.0,230.0,428.0,293.0
4,0.0,2008_000009,10,217.0,161.0,294.0,221.0


In [18]:
def find_intersection(set_1, set_2):
    """
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.
    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)


def find_jaccard_overlap(set_1, set_2):
    """
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.
    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)


# Some augmentation functions below have been adapted from
# From https://github.com/amdegroot/ssd.pytorch/blob/master/utils/augmentations.py

def expand(image, boxes, filler):
    """
    Perform a zooming out operation by placing the image in a larger canvas of filler material.
    Helps to learn to detect smaller objects.
    :param image: image, a tensor of dimensions (3, original_h, original_w)
    :param boxes: bounding boxes in boundary coordinates, a tensor of dimensions (n_objects, 4)
    :param filler: RBG values of the filler material, a list like [R, G, B]
    :return: expanded image, updated bounding box coordinates
    """
    # Calculate dimensions of proposed expanded (zoomed-out) image
    original_h = image.size(1)
    original_w = image.size(2)
    max_scale = 4
    scale = random.uniform(1, max_scale)
    new_h = int(scale * original_h)
    new_w = int(scale * original_w)

    # Create such an image with the filler
    filler = torch.FloatTensor(filler)  # (3)
    new_image = torch.ones((3, new_h, new_w), dtype=torch.float) * filler.unsqueeze(1).unsqueeze(1)  # (3, new_h, new_w)
    # Note - do not use expand() like new_image = filler.unsqueeze(1).unsqueeze(1).expand(3, new_h, new_w)
    # because all expanded values will share the same memory, so changing one pixel will change all

    # Place the original image at random coordinates in this new image (origin at top-left of image)
    left = random.randint(0, new_w - original_w)
    right = left + original_w
    top = random.randint(0, new_h - original_h)
    bottom = top + original_h
    new_image[:, top:bottom, left:right] = image

    # Adjust bounding boxes' coordinates accordingly
    new_boxes = boxes + torch.FloatTensor([left, top, left, top]).unsqueeze(
        0)  # (n_objects, 4), n_objects is the no. of objects in this image

    return new_image, new_boxes


def random_crop(image, boxes, labels, difficulties):
    """
    Performs a random crop in the manner stated in the paper. Helps to learn to detect larger and partial objects.
    Note that some objects may be cut out entirely.
    Adapted from https://github.com/amdegroot/ssd.pytorch/blob/master/utils/augmentations.py
    :param image: image, a tensor of dimensions (3, original_h, original_w)
    :param boxes: bounding boxes in boundary coordinates, a tensor of dimensions (n_objects, 4)
    :param labels: labels of objects, a tensor of dimensions (n_objects)
    :param difficulties: difficulties of detection of these objects, a tensor of dimensions (n_objects)
    :return: cropped image, updated bounding box coordinates, updated labels, updated difficulties
    """
    original_h = image.size(1)
    original_w = image.size(2)
    # Keep choosing a minimum overlap until a successful crop is made
    while True:
        # Randomly draw the value for minimum overlap
        min_overlap = random.choice([0., .1, .3, .5, .7, .9, None])  # 'None' refers to no cropping

        # If not cropping
        if min_overlap is None:
            return image, boxes, labels, difficulties

        # Try up to 50 times for this choice of minimum overlap
        # This isn't mentioned in the paper, of course, but 50 is chosen in paper authors' original Caffe repo
        max_trials = 50
        for _ in range(max_trials):
            # Crop dimensions must be in [0.3, 1] of original dimensions
            # Note - it's [0.1, 1] in the paper, but actually [0.3, 1] in the authors' repo
            min_scale = 0.3
            scale_h = random.uniform(min_scale, 1)
            scale_w = random.uniform(min_scale, 1)
            new_h = int(scale_h * original_h)
            new_w = int(scale_w * original_w)

            # Aspect ratio has to be in [0.5, 2]
            aspect_ratio = new_h / new_w
            if not 0.5 < aspect_ratio < 2:
                continue

            # Crop coordinates (origin at top-left of image)
            left = random.randint(0, original_w - new_w)
            right = left + new_w
            top = random.randint(0, original_h - new_h)
            bottom = top + new_h
            crop = torch.FloatTensor([left, top, right, bottom])  # (4)

            # Calculate Jaccard overlap between the crop and the bounding boxes
            overlap = find_jaccard_overlap(crop.unsqueeze(0),
                                           boxes)  # (1, n_objects), n_objects is the no. of objects in this image
            overlap = overlap.squeeze(0)  # (n_objects)

            # If not a single bounding box has a Jaccard overlap of greater than the minimum, try again
            if overlap.max().item() < min_overlap:
                continue

            # Crop image
            new_image = image[:, top:bottom, left:right]  # (3, new_h, new_w)

            # Find centers of original bounding boxes
            bb_centers = (boxes[:, :2] + boxes[:, 2:]) / 2.  # (n_objects, 2)

            # Find bounding boxes whose centers are in the crop
            centers_in_crop = (bb_centers[:, 0] > left) * (bb_centers[:, 0] < right) * (bb_centers[:, 1] > top) * (
                    bb_centers[:, 1] < bottom)  # (n_objects), a Torch uInt8/Byte tensor, can be used as a boolean index

            # If not a single bounding box has its center in the crop, try again
            if not centers_in_crop.any():
                continue

            # Discard bounding boxes that don't meet this criterion
            new_boxes = boxes[centers_in_crop, :]
            new_labels = labels[centers_in_crop]
            new_difficulties = difficulties[centers_in_crop]

            # Calculate bounding boxes' new coordinates in the crop
            new_boxes[:, :2] = torch.max(new_boxes[:, :2], crop[:2])  # crop[:2] is [left, top]
            new_boxes[:, :2] -= crop[:2]
            new_boxes[:, 2:] = torch.min(new_boxes[:, 2:], crop[2:])  # crop[2:] is [right, bottom]
            new_boxes[:, 2:] -= crop[:2]

            return new_image, new_boxes, new_labels, new_difficulties


def flip(image, boxes):
    """
    Flip image horizontally.
    :param image: image, a PIL Image
    :param boxes: bounding boxes in boundary coordinates, a tensor of dimensions (n_objects, 4)
    :return: flipped image, updated bounding box coordinates
    """
    # Flip image
    new_image = FT.hflip(image)

    # Flip boxes
    new_boxes = boxes
    new_boxes[:, 0] = image.width - boxes[:, 0] - 1
    new_boxes[:, 2] = image.width - boxes[:, 2] - 1
    new_boxes = new_boxes[:, [2, 1, 0, 3]]

    return new_image, new_boxes


def resize(image, boxes, dims=(300, 300), return_percent_coords=True):
    """
    Resize image. For the SSD300, resize to (300, 300).
    Since percent/fractional coordinates are calculated for the bounding boxes (w.r.t image dimensions) in this process,
    you may choose to retain them.
    :param image: image, a PIL Image
    :param boxes: bounding boxes in boundary coordinates, a tensor of dimensions (n_objects, 4)
    :return: resized image, updated bounding box coordinates (or fractional coordinates, in which case they remain the same)
    """
    # Resize image
    new_image = FT.resize(image, dims)

    # Resize bounding boxes
    old_dims = torch.FloatTensor([image.width, image.height, image.width, image.height]).unsqueeze(0)
    new_boxes = boxes / old_dims  # percent coordinates

    if not return_percent_coords:
        new_dims = torch.FloatTensor([dims[1], dims[0], dims[1], dims[0]]).unsqueeze(0)
        new_boxes = new_boxes * new_dims

    return new_image, new_boxes


def photometric_distort(image):
    """
    Distort brightness, contrast, saturation, and hue, each with a 50% chance, in random order.
    :param image: image, a PIL Image
    :return: distorted image
    """
    new_image = image

    distortions = [FT.adjust_brightness,
                   FT.adjust_contrast,
                   FT.adjust_saturation,
                   FT.adjust_hue]

    random.shuffle(distortions)

    for d in distortions:
        if random.random() < 0.5:
            if d.__name__ is 'adjust_hue':
                # Caffe repo uses a 'hue_delta' of 18 - we divide by 255 because PyTorch needs a normalized value
                adjust_factor = random.uniform(-18 / 255., 18 / 255.)
            else:
                # Caffe repo uses 'lower' and 'upper' values of 0.5 and 1.5 for brightness, contrast, and saturation
                adjust_factor = random.uniform(0.5, 1.5)

            # Apply this distortion
            new_image = d(new_image, adjust_factor)

    return new_image


def transform(image, boxes, labels, difficulties, split):
    """
    Apply the transformations above.
    :param image: image, a PIL Image
    :param boxes: bounding boxes in boundary coordinates, a tensor of dimensions (n_objects, 4)
    :param labels: labels of objects, a tensor of dimensions (n_objects)
    :param difficulties: difficulties of detection of these objects, a tensor of dimensions (n_objects)
    :param split: one of 'TRAIN' or 'TEST', since different sets of transformations are applied
    :return: transformed image, transformed bounding box coordinates, transformed labels, transformed difficulties
    """
    assert split in {'TRAIN', 'TEST'}

    # Mean and standard deviation of ImageNet data that our base VGG from torchvision was trained on
    # see: https://pytorch.org/docs/stable/torchvision/models.html
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    new_image = image
    new_boxes = boxes
    new_labels = labels
    new_difficulties = difficulties
    # Skip the following operations for evaluation/testing
    if split == 'TRAIN':
        # A series of photometric distortions in random order, each with 50% chance of occurrence, as in Caffe repo
        new_image = photometric_distort(new_image)

        # Convert PIL image to Torch tensor
        new_image = FT.to_tensor(new_image)

        # Expand image (zoom out) with a 50% chance - helpful for training detection of small objects
        # Fill surrounding space with the mean of ImageNet data that our base VGG was trained on
        if random.random() < 0.5:
            new_image, new_boxes = expand(new_image, boxes, filler=mean)

        # Randomly crop image (zoom in)
        new_image, new_boxes, new_labels, new_difficulties = random_crop(new_image, new_boxes, new_labels,
                                                                         new_difficulties)

        # Convert Torch tensor to PIL image
        new_image = FT.to_pil_image(new_image)

        # Flip image with a 50% chance
        if random.random() < 0.5:
            new_image, new_boxes = flip(new_image, new_boxes)

    # Resize image to (300, 300) - this also converts absolute boundary coordinates to their fractional form
    new_image, new_boxes = resize(new_image, new_boxes, dims=(300, 300))

    # Convert PIL image to Torch tensor
    new_image = FT.to_tensor(new_image)

    # Normalize by mean and standard deviation of ImageNet data that our base VGG was trained on
    new_image = FT.normalize(new_image, mean=mean, std=std)

    return new_image, new_boxes, new_labels, new_difficulties


def adjust_learning_rate(optimizer, scale):
    """
    Scale learning rate by a specified factor.
    :param optimizer: optimizer whose learning rate must be shrunk.
    :param scale: factor to multiply learning rate with.
    """
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * scale
    print("DECAYING learning rate.\n The new LR is %f\n" % (optimizer.param_groups[1]['lr'],))

In [19]:
class VOCDataset(Dataset):
    
    def __init__(self, dataframe, image_dict, split, keep_difficult=False):
        super().__init__()
        
        self.image_ids = dataframe['img_id'].unique()
        self.df = dataframe
        self.image_dict = image_dict
        self.transforms = transforms
        self.split = split
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['img_id'] == image_id]
        
        image = self.image_dict[image_id]
        image = image.convert("RGB")
        
        boxes = records[['xmin', 'ymin', 'xmax','ymax']].values
        boxes = torch.FloatTensor(records[['xmin', 'ymin', 'xmax','ymax']].values)
        
        labels = records['labels'].values
        labels = torch.as_tensor(labels, dtype=torch.int64)

        difficulties = records["difficulties"].values
        difficulties = torch.as_tensor(difficulties, dtype=torch.int32)
        
#         boxes = self.box_resize(boxes, image)
        seed = np.int64(42) # make a seed with numpy generator 
        random.seed(seed)
        torch.manual_seed(seed)
        # Apply transformations
        image, boxes, labels, difficulties = transform(image, boxes, labels, difficulties, split=self.split) 
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor(float(image_id), dtype=torch.float32)
        target["difficulties"] = difficulties
        
        return image, target
    

    def box_resize(self, box, img, dims=(300, 300)):
        old_dims = torch.FloatTensor([img.width, img.height, img.width, img.height]).unsqueeze(0)
        new_box = box / old_dims
        new_dims = torch.FloatTensor([dims[1], dims[0], dims[1], dims[0]]).unsqueeze(0)        
        return new_box   
    
    def __len__(self) -> int:
        return self.image_ids.shape[0]
    
    def collate_fn(self, batch):
        """
        Since each image may have a different number of objects, we need a collate function (to be passed to the DataLoader).
        This describes how to combine these tensors of different sizes. We use lists.
        Note: this need not be defined in this Class, can be standalone.
        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
        """
        images = list()
        boxes = list()
        labels = list()
        difficulties = list()

        for b in batch:
            images.append(b[0])
            boxes.append(b[1])
            labels.append(b[2])
            difficulties.append(b[3])

        images = torch.stack(images, dim=0)

        return images, boxes, labels, difficulties  # tensor (N, 3, 300, 300), 3 lists of N tensors each

In [20]:
df_train.head()

,difficulties,img_id,labels,xmin,ymin,xmax,ymax
0,0.0,2008_000008,13,53.0,87.0,471.0,420.0
1,0.0,2008_000008,15,158.0,44.0,289.0,167.0
2,0.0,2008_000015,5,270.0,1.0,378.0,176.0
3,0.0,2008_000015,5,57.0,1.0,164.0,150.0
4,0.0,2008_000019,12,139.0,2.0,372.0,197.0


In [21]:
keep_difficult = True
train_dataset = VOCDataset(df_train, train_image_dict, "TRAIN",keep_difficult)
test_dataset = VOCDataset(df_test, test_image_dict, "TEST",keep_difficult)

del train_image_dict, test_image_dict
del df_train, df_test

In [22]:
import utils

In [23]:
# split the dataset in train and test set
train_data_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=utils.collate_fn
)

test_data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=utils.collate_fn
)

In [24]:
print("Number of training iterations: ", len(train_data_loader))
print("Number of testing iterations: ", len(test_data_loader))

Number of training iterations:  358
Number of testing iterations:  5823


In [25]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = False)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = len(label_map)  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [26]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_frcnn.pth.tar'
    torch.save(state, filename)

In [27]:
from engine import train_one_epoch, evaluate

# Data parameters
data_folder = '/kaggle/input/d/tungbinhthuong/pascal-voc-2012/'  # folder with data files

# Model parameters
# Not too many here since the SSD300 has a very specific structure
n_classes = len(label_map)  # number of different types of objects
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Learning parameters
checkpoint = '/kaggle/working/checkpoint_frcnn_resnet.pth.tar'
  # path to model checkpoint, None if none
# batch_size = 16  # batch size
iterations = 120000  # number of iterations to train
# workers = 4  # number of workers for loading data in the DataLoader
print_freq = 50  # print training status every __ batches
lr = 1e-3  # learning rate
decay_lr_at = [80000, 100000]  # decay learning rate after these many iterations
decay_lr_to = 0.1  # decay learning rate to this fraction of the existing learning rate
momentum = 0.9  # momentum
weight_decay = 5e-4  # weight decay
grad_clip = True  # clip if gradients are exploding, which may happen at larger batch sizes (sometimes at 32) - you will recognize it by a sorting error in the MuliBox loss calculation

def train(model, data_loader, data_loader_test):
    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 100 epochs
    num_epochs = 1

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        gc.collect()
        torch.cuda.empty_cache()
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
#         evaluate(model, data_loader_test, device=device)
        save_checkpoint(epoch, model, optimizer)
    print("That's it!")

In [28]:
train(model, train_data_loader, test_data_loader)

Epoch: [0]  [  0/358]  eta: 0:51:35  lr: 0.000019  loss: 4.8191 (4.8191)  loss_classifier: 3.1017 (3.1017)  loss_box_reg: 0.0008 (0.0008)  loss_objectness: 0.7169 (0.7169)  loss_rpn_box_reg: 0.9996 (0.9996)  time: 8.6475  data: 0.3189  max mem: 11294
Epoch: [0]  [ 10/358]  eta: 0:15:44  lr: 0.000159  loss: 4.2899 (3.9398)  loss_classifier: 2.3939 (2.1382)  loss_box_reg: 0.0011 (0.0016)  loss_objectness: 0.7108 (0.7068)  loss_rpn_box_reg: 1.0709 (1.0932)  time: 2.7153  data: 0.3418  max mem: 11558
Epoch: [0]  [ 20/358]  eta: 0:13:47  lr: 0.000299  loss: 2.3835 (3.0858)  loss_classifier: 0.6490 (1.3353)  loss_box_reg: 0.0023 (0.0034)  loss_objectness: 0.6759 (0.6319)  loss_rpn_box_reg: 1.1194 (1.1152)  time: 2.1372  data: 0.3573  max mem: 11558
Epoch: [0]  [ 30/358]  eta: 0:12:45  lr: 0.000439  loss: 1.5268 (2.5214)  loss_classifier: 0.2405 (0.9672)  loss_box_reg: 0.0097 (0.0078)  loss_objectness: 0.4131 (0.5386)  loss_rpn_box_reg: 0.9071 (1.0078)  time: 2.1251  data: 0.3475  max mem: 11

KeyboardInterrupt: 

In [29]:
model.eval()
for images, targets in train_data_loader:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    scaler = None
    with torch.cuda.amp.autocast(enabled=scaler is not None):
        loss_dict = model(images, targets)
        print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())
    print(loss_dict)
    break

RuntimeError: CUDA out of memory. Tried to allocate 626.00 MiB (GPU 0; 15.90 GiB total capacity; 13.98 GiB already allocated; 151.75 MiB free; 14.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF